# Pascal segmentation

The Pascal VOC challenge is a very popular dataset for building and evaluating algorithms for image classification, object detection, and segmentation.

Classes:
`background`, `aeroplane`, `bicycle`, `bird`, `boat`, `bottle`, `bus`, `car`, `cat`, `chair`, `cow`, `diningtable`, `dog`, `horse`, `motorbike`, `person`, `pottedplant`, `sheep`, `sofa`, `train`, `tvmonitor`

255 is the ignore label that marks pixels excluded from learning and
evaluation by the PASCAL VOC ground truth.

#### 1. Necessary imports

In [ ]:
import sys

import PIL
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append('../batchflow')
from batchflow import B, V, F, R, P, W
from batchflow.opensets import PascalSegmentation
from batchflow.models.torch import TorchModel, UNet
from batchflow.models.metrics import ClassificationMetrics

plt.style.use('seaborn-poster')
plt.style.use('ggplot')

#### 2. Load data

In [ ]:
ds = PascalSegmentation(bar=True)

Let's check few images

In [ ]:
# We should construct batch and get images.
batch = ds.train.p.next_batch(16)
images = batch.images
labels = batch.labels

In [ ]:
images[2]

In [ ]:
labels[2]

#### 3. Define training pipeline

In [ ]:
NUM_CLASSES = 22
model_config = {
    'inputs/targets/classes': NUM_CLASSES,
}

In [ ]:
# Define constants
BATCH_SIZE = 16
N_EPOCHS = 1
SIZE = (160, 160)

In [ ]:
def process_mask(x):
    x = np.squeeze(x)
    np.place(x, x==255, 21)
    return x

In [ ]:
# Define actions for training
train_ppl = (ds.train.p
    .init_model('dynamic', UNet, 'model', config=model_config)
    .init_variable('loss', [])
    .resize(size=SIZE, src='images', dst='images')
    .resize(size=SIZE, src='labels', dst='labels')
    .to_array(channels='first', src='images', dst='images')
    .to_array(channels='first', src='labels', dst='labels')
    .apply_transform_all(src='labels', dst='labels', func=process_mask)
    .train_model('model', B('images'), B('labels'), fetches='loss', save_to=V('loss', mode='a'))
    .run_later(BATCH_SIZE, n_epochs=10, drop_last=True, shuffle=42, bar='n')
)

In [ ]:
# Here we will run it
train_ppl.run()

#### 4. ... and test pipeline

In [ ]:
# Define actions for test
test_ppl = (ds.test.p
    .import_model('model', train_ppl)
    .init_variable('metrics', None)
    .resize(size=SIZE, src='images', dst='images')
    .resize(size=SIZE, src='labels', dst='labels')
    .to_array(channels='first', src='images', dst='images2')
    .to_array(channels='first', src='labels', dst='labels2')
    .apply_transform_all(src='labels', dst='labels', func=process_mask)
    .predict_model('model', B('images2'), fetches='predictions', save_to=B('predictions'))
    .gather_metrics('classification', B('labels2'), B('predictions'), axis=1, fmt='logits',
                    num_classes=NUM_CLASSES, save_to=V('metrics', mode='u'))
)

In [ ]:
test_ppl.run(4, drop_last=False, bar=True)

#### 5. Evaluate metrics

In [ ]:
f1_scores = test_ppl.v('metrics').evaluate('f1_score', agg='mean', multiclass=None)
mean_f1_score = test_ppl.v('metrics').evaluate('f1_score')

mean_f1_score

#### Task:

Try to modify the model and training process to get better `mean_f1_score`.

#### What you can change?
For example, you can add additional keys into dictionary and then try to vary them:

```
model_config = {
                'inputs/targets/classes': NUM_CLASSES,
                'body/encoder/num_stages': 4,
                'body/decoder/blocks/filters': [128, 64, 64, 32],
                'body/encoder/blocks/filters': [32, 64, 64, 128],
                'body/embedding/filters': 128,
                'head': dict(layout='c', filters=NUM_CLASSES, kernel_size=1),
    
                'optimizer': ('Adam', {'lr': 0.001}),
            }
```